## Import Data

In [1]:
# Setup Packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

%matplotlib inline

# Display all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [32]:
house_data = pd.read_csv('train.csv')
# house_data = pd.read_csv('test.csv')

remove_outlier = 1  # change to 0 for test data

<IPython.core.display.Javascript object>

## Data Manipulation

### Response Variable - Log Transform
- Transformation validated in DataExploration Notebook

In [33]:
house_data['SalePrice_log'] = np.log(house_data['SalePrice'])

<IPython.core.display.Javascript object>

### Impute Continuous Variables
- Imputation with mean validated in DataExploration Notebook

In [35]:
house_data["LotFrontage"].mean()

70.04995836802642

<IPython.core.display.Javascript object>

In [36]:
# LotFrontage
avg_LotFrontage = house_data['LotFrontage'].mean()
house_data['LotFrontage'].fillna(avg_LotFrontage, inplace=True)

# MasVnrArea
avg_LotFrontage = house_data['MasVnrArea'].mean()
house_data['MasVnrArea'].fillna(avg_LotFrontage, inplace=True)


<IPython.core.display.Javascript object>

### Impute Categorical Variables

In [5]:
#Impute NA values with 'None'
house_data['BsmtQual'].fillna('None', inplace=True)
house_data['BsmtCond'].fillna('None', inplace=True)
house_data['BsmtExposure'].fillna('None', inplace=True)
house_data['BsmtFinType1'].fillna('None', inplace=True)
train_data_cat_var.at[948,'BsmtExposure']='No'

<IPython.core.display.Javascript object>

In [6]:
#Electrical Fill with Mode
house_data['Electrical'].fillna(house_data['Electrical'].mode()[0], inplace=True)

<IPython.core.display.Javascript object>

In [7]:
#Fire Place Quality
house_data['FireplaceQu'].fillna('None', inplace=True)

<IPython.core.display.Javascript object>

In [9]:
#Garage Related Variables
house_data['GarageType'].fillna('None', inplace=True)
house_data['GarageFinish'].fillna('None', inplace=True)
house_data['GarageQual'].fillna('None', inplace=True)

<IPython.core.display.Javascript object>

In [ ]:
house_data['PoolQC'].fillna("None", inplace=True)

In [ ]:
house_data['Fence'].fillna("None", inplace=True)

In [ ]:
house_data['MiscFeature'].fillna("None", inplace=True)

## Feature Engineering

In [10]:
#Total area of house = Above ground living area + basement living area
house_data['TotalSF'] = house_data['GrLivArea'] + house_data['TotalBsmtSF']

<IPython.core.display.Javascript object>

In [11]:
#Age = Year sold - year built
house_data['Age'] = house_data['YrSold'] + house_data['YearBuilt']

<IPython.core.display.Javascript object>

In [12]:
#AgeRemod = Year sold - year remodeled
house_data['Age'] = house_data['YrSold'] + house_data['YearRemodAdd']

<IPython.core.display.Javascript object>

In [13]:
#Total porch SF = OpenPorchSF + EnclosedSF + 3SsnPorch + ScreenPorch
house_data['TotPorchSF'] = house_data['OpenPorchSF'] + house_data['EnclosedPorch'] + house_data['3SsnPorch'] + house_data['ScreenPorch']

<IPython.core.display.Javascript object>

In [14]:
#Total bathrooms = Full bath + 0.5*Halfbath (for both house and basement)
house_data['TotBaths'] = house_data['FullBath'] + house_data['BsmtFullBath'] + 0.5*house_data['HalfBath'] + 0.5*house_data['BsmtHalfBath']

<IPython.core.display.Javascript object>

In [15]:
#Total number of car garage = 1, 2, 3+ cars
house_data['TotCarGarage'] = pd.Series(len(house_data['GarageCars']), index=house_data.index)
house_data['TotCarGarage'] = 0 
house_data.loc[house_data['GarageCars'] >= 3,'TotCarGarage'] = 1

/Users/jonathanharris/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


<IPython.core.display.Javascript object>

In [16]:
#Has Second Floor = 0 (no) or 1 (yes)
house_data['Has2ndFl'] = pd.Series(len(house_data['2ndFlrSF']), index=house_data.index)
house_data['Has2ndFl'] = 0 
house_data.loc[house_data['2ndFlrSF'] > 0,'Has2ndFl'] = 1


<IPython.core.display.Javascript object>

In [18]:
#Has Porch = 0 (no) or 1 (yes)
house_data['HasPorch'] = pd.Series(len(house_data['TotPorchSF']), index=house_data.index)
house_data['HasPorch'] = 0 
house_data.loc[house_data['TotPorchSF'] > 0,'HasPorch'] = 1


<IPython.core.display.Javascript object>

In [19]:
#Has Deck = 0 (no) or 1 (yes)
house_data['HasDeck'] = pd.Series(len(house_data['WoodDeckSF']), index=house_data.index)
house_data['HasDeck'] = 0 
house_data.loc[house_data['WoodDeckSF'] > 0,'HasDeck'] = 1

<IPython.core.display.Javascript object>

In [20]:
#Has Pool = 0 (no) or 1 (yes)
house_data['HasPool'] = pd.Series(len(house_data['PoolArea']), index=house_data.index)
house_data['HasPool'] = 0 
house_data.loc[house_data['PoolArea'] > 0,'HasPool'] = 1

<IPython.core.display.Javascript object>

In [21]:
#Has Fireplace = 0 (no), 1 (yes, not excellent quality), or 2 (excellent quality)
house_data['HasFirePlace'] = pd.Series(len(house_data['FireplaceQu']), index=house_data.index)
house_data['HasFirePlace'] = 1 #default all houses have a fireplace 
house_data.loc[house_data['FireplaceQu'].isna(),'HasFirePlace'] = 0 #replace 1 with 0 for 'No Fireplace'
house_data.loc[house_data['FireplaceQu']== 'Ex','HasFirePlace'] = 2 #replace 1 with 2 for 'Excellent Fireplace'

<IPython.core.display.Javascript object>

In [22]:
#Has Gas Heating = 0 (no) or 1 (yes, GasA or GasW)
house_data['HasGasHeating'] = pd.Series(len(house_data['Heating']), index=house_data.index)
house_data['HasGasHeating'] = 0 
house_data.loc[house_data['Heating']== 'GasA','HasGasHeating'] = 1
house_data.loc[house_data['Heating']== 'GasW','HasGasHeating'] = 1

<IPython.core.display.Javascript object>

In [24]:
#Has Modern Circuitbreaker = 0 (no) or 1 (yes)
house_data['HasSBrkr'] = pd.Series(len(house_data['Electrical']), index=house_data.index)
house_data['HasSBrkr'] = 0 
house_data.loc[house_data['Electrical']== 'SBrkr','HasSBrkr'] = 1

<IPython.core.display.Javascript object>

In [25]:
#Has excellent basement quality = 0 (no) or 1 (yes)
house_data['HasExBsmtQual'] = pd.Series(len(house_data['BsmtQual']), index=house_data.index)
house_data['HasExBsmtQual'] = 0 
house_data.loc[house_data['BsmtQual']== 'Ex','HasExBsmtQual'] = 1

<IPython.core.display.Javascript object>

In [26]:
#Has Basement Exposure = 0 (no) or 1 (yes)
house_data['HasGd'] = pd.Series(len(house_data['BsmtExposure']), index=house_data.index)
house_data['HasGd'] = 0 
house_data.loc[house_data['BsmtExposure']== 'Gd','HasGd'] = 1

<IPython.core.display.Javascript object>

In [27]:
#Has Basement with good living quarters = 0 (no) or 1 (yes)
house_data['HasGLQ'] = pd.Series(len(house_data['BsmtFinType1']), index=house_data.index)
house_data['HasGLQ'] = 0 
house_data.loc[house_data['BsmtFinType1']== 'GLQ','HasGLQ'] = 1

<IPython.core.display.Javascript object>

In [28]:
#Has paved driveway = 0 (no) or 1 (yes)
house_data['HasPavedDrive'] = pd.Series(len(house_data['PavedDrive']), index=house_data.index)
house_data['HasPavedDrive'] = 0 
house_data.loc[house_data['PavedDrive']== 'Y','HasPavedDrive'] = 1

<IPython.core.display.Javascript object>

In [ ]:
house_data['HasTA'] = pd.Series(len(house_data['GarageQual']), index=house_data.index)
house_data['HasTA'] = 0 
house_data.loc[house_data['GarageQual']== 'TA','HasTA'] = 1

## Take stock of columns 

In [ ]:
# house_data has all of the columns
# cat_var_all is subset 
# drop_cat_var_all

In [2]:
cat_var_all= ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
              'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 
              'PoolQC', 'Fence', 'MiscFeature','SaleType', 'SaleCondition', 'SalePrice_log', 'SalePrice']

drop_cat_var_all= ['Heating','Electrical', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'PavedDrive', 'GarageQual']
cat_only_drop=['Fence', 'MiscFeature', 'SaleType']
ordinal_drop=['BsmtFinType2','Functional', 'PoolQC', 'GarageCond', 'SaleCondition']




## One hot encode 

In [ ]:
'''
def Diff(li1, li2): 
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2] 
    return li_dif 

train_data_cat_var_final= []
cols= Diff(list(train_data_cat_var_final.columns),['SalePrice_log', 'SalePrice'])

house_data_subset = pd.get_dummies(house_data[cols], prefix=cols, columns=cols, drop_first=True)

## Remove Outliers

In [30]:
# # Separate predictors by numerical and categorical features
# from functions_file import categorize

# cat_features, num_cont_features, num_disc_features = categorize(house_data)

# # Determine Outliers of Continuous Data
# def outlier_idx(data, thresh):

#     avg = data.mean()  # calc average
#     stdev = data.std()  # calc standard deviation
#     z_score = (data - avg) / stdev  # calc z_score
#     ol = z_score > thresh  # boolean (True=outlier)
#     outlier_idx = data.index[ol]  # indexes of outliers

#     return list(outlier_idx)


# # Plot Outliers
# plt.figure(figsize=(15, 20))
# for idx, col in enumerate(num_cont_features):
#     outliers = outlier_idx(house_data[col], 6)  # outlier indices
#     plt.subplot(math.ceil(len(num_cont_features) / ncol), ncol, idx + 1)
#     plt.scatter(x=house_data[col], y=house_data["SalePrice_log"], c="blue")
#     plt.scatter(
#         x=house_data[col][outliers], y=house_data["SalePrice_log"][outliers], c="red"
#     )
#     plt.ylabel("SalePrice_log")
#     plt.xlabel(col)

NameError: name 'plt' is not defined

<IPython.core.display.Javascript object>

In [ ]:
# vars_outliers = 'LotFrontage', 'LotArea', 'MasVnrArea', 'TotalBsmtSF', 'GrLivArea'

# outlier_store = {}
# thresh = 6  # outlier > +6std away from mean

# for idx, col in enumerate(num_cont_features):
#     outliers = outlier_idx(house_data[col], thresh)  # outlier indices

#     if len(outliers) > 0:
#         outlier_store[col] = outliers  # only store features with outliers

## Remove Variables

## Consolidate Data Feature Engineered: 

In [42]:
house_data = pd.read_csv('csv')
house_data.shape

(1460, 23)

In [45]:
house_data2= pd.read_csv('train1_kc.csv')
house_data2.shape

In [46]:
Id= np.array(house_data['Id'])-1
house_data['Id']= Id
house_data= house_data.drop(['SalePrice', 'SalePrice_log'], axis=1)

In [53]:
train_all=pd.merge(house_data, house_data2, how= 'left', left_on= 'Id', right_on= house_data2.index)

In [43]:
house_data.columns

Index(['Unnamed: 0', 'Id', 'MSZoning', 'Street', 'Alley', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
       'ExterCond', 'Foundation'],
      dtype='object')

In [58]:
train_all=train_all.drop(['Unnamed: 0_y', 'Unnamed: 0_x'], axis=1)

In [62]:
test.to_csv('train_all.csv')

In [ ]:
cont_var_JH= ['Id','LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF',
'1stFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageYrBlt', 'GarageArea', 'WoodDeckSF',
'MiscVal','TotalSF', 'Age', 'TotPorchSF', 'TotBaths', 'MoSold', 'TotCarGarage']

cat_var_JH= ['Has2ndFl','NeighborhoodType','HasPorch', 'HasDeck', 'HasPool'] 

## Consolidated DATA  Train_ Test 

In [45]:
jon= pd.read_csv('jon_original_HouseData.csv')
rich = pd.read_csv('rk_hotencode.csv')
kailun= pd.read_csv('train_all0.csv')

In [90]:
jon_test= pd.read_csv('jon_test_HouseData.csv')
rich_test = pd.read_csv('rk_hotencode_test.csv')
kailun_test= pd.read_csv('test_all0.csv')

In [100]:
# Examine Data 
kailun_test.head()

,Id,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,KitchenQual,Functional,FireplaceQu,GarageFinish,GarageCond,Fence,HasGasHeating,Electrical_combined,GarageType_combined,GarageQual_combined,Garage_combined,SaleType_combined,HasPavedDrive,HasPavedDrive_combined,SaleCond_combined,HasCentralAir
0,0,4,4,2,4,3,3,3,8,1,2,4,4,1,1,1,2,3.0,1,0,1.0,1,1
1,1,4,4,2,6,2,3,4,8,1,2,4,1,1,1,1,2,3.0,1,0,1.0,1,1
2,2,5,4,2,7,2,4,3,8,4,4,4,4,1,1,1,2,3.0,1,0,1.0,1,1
3,3,4,4,2,7,2,5,4,8,5,4,4,1,1,1,1,2,3.0,1,0,1.0,1,1
4,4,5,4,2,6,2,5,4,8,1,3,4,1,1,1,1,2,3.0,1,0,1.0,1,1


In [98]:
rich_test= rich_test.drop('Unnamed: 0', axis=1)
jon_test= jon_test.drop('Unnamed: 0', axis=1)

In [77]:
# Note this is not needed for Test data set since there are no outliers 
Id= np.array(jon['Id'])-1
jon['Id']= Id
#house_data= house_data.drop(['SalePrice', 'SalePrice_log'], axis=1)
jon= jon.drop(['Unnamed: 0'], axis=1)

In [63]:
train_all_cat=pd.merge(kailun, rich, how= 'left', left_on= 'Id', right_on= rich.index)

In [101]:
test_all_cat=pd.merge(jon_test, rich_test, how= 'left', left_on= 'Id', right_on= 'Id')

In [65]:
drop=['Id_x', 'Id_y', 'Unnamed: 0', ]
train_all_cat= train_all_cat.drop('Unnamed: 0', axis=1)

In [79]:
train_all_final= pd.merge(jon, train_all_cat, how='left', left_on= 'Id', right_on= 'Id')

In [104]:
test_all_final=pd.merge(test_all_cat, kailun_test, how= 'left', left_on= 'Id', right_on= 'Id')

In [86]:
drop=['SalePrice_x','SalePrice_log_x']
train_all_final=train_all_final.drop(drop, axis=1)

In [105]:
test_all_final.head()

,Id,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,KitchenQual,Functional,FireplaceQu,GarageFinish,GarageCond,Fence,HasGasHeating,Electrical_combined,GarageType_combined,GarageQual_combined,Garage_combined,SaleType_combined,HasPavedDrive,HasPavedDrive_combined,SaleCond_combined,HasCentralAir
0,1461,80.0,11622,5,6,1961,1961,0.0,468,144,270,882,896,0,0,896,0,0,1,0,2,1,5,0,1961,1,730,140,0,0,0,120,0,0,6,2010,3.0,2,3,1,4,1.0,5,1,13,2,3,1,3,2,1,11.0,14.0,3,3,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1462,81.0,14267,6,6,1958,1958,108.0,923,0,406,1329,1329,0,0,1329,0,0,1,1,3,1,6,0,1958,1,312,393,36,0,0,0,0,12500,6,2010,4.0,2,3,2,4,1.0,1,1,13,3,3,1,3,4,1,12.0,15.0,2,3,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1463,74.0,13830,5,5,1997,1998,0.0,791,0,137,928,928,701,0,1629,0,0,2,1,3,1,6,1,1997,2,482,212,34,0,0,0,0,0,3,2010,4.0,2,3,2,4,1.0,5,1,9,3,3,1,5,2,1,11.0,14.0,3,3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1464,78.0,9978,6,6,1998,1998,20.0,602,0,324,926,926,678,0,1604,0,0,2,1,3,1,7,1,1998,2,470,360,36,0,0,0,0,0,6,2010,4.0,2,3,2,4,1.0,5,1,9,3,3,1,5,2,1,11.0,14.0,2,3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1465,43.0,5005,8,5,1992,1992,0.0,263,0,1017,1280,1280,0,0,1280,0,0,2,0,2,1,5,0,1992,2,506,0,82,0,0,144,0,0,1,2010,4.0,2,3,2,2,1.0,5,1,22,3,3,5,3,2,1,7.0,8.0,3,4,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
cat_var_RK= ['Alley','BldgType','Exterior1st', 'Condition1', 'Condition2', 'Exterior2nd', 'Foundation', 'HouseStyle', 
             'LandContour','LotConfig', 'MasVnrType', 'MSSubClass', 'MSZoning', 'RoofMatl', 'Street', 'Utilities']
ord_var_RK= ['ExterCond', 'ExterQual', 'LandSlope', 'LotShape']

In [ ]:
cat_var_KC= ['HasGasHeating', 'Electrical_combined', 'CentralAir_Y', 'GarageType_Combined', 
             'GarageQual_combined','HasPavedDrive', 'SaleType_combined', 'SalesCond_Combined']
ord_var_KC= ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC',
            'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageCond', 'Fence']

In [ ]:
cat_var_JH= ['MoSold','YrSold', 'GarageYrBlt']

In [88]:
train_all_final.to_csv('train_all_final.csv')

In [106]:
test_all_final.to_csv('test_all_final.csv')